In [ ]:
import pandas as pd
from agent.text_agent import run_from_text

df = pd.read_csv("../data/model_data_nf_v1.csv")

result = run_from_text(
    df,
    "Test whether higher annual mileage leads to higher loss ratio"
)

result["decision"]
